# Final Project

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import time

from keras.wrappers.scikit_learn import KerasRegressor
from typing import Final
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # Suppress TF AVX info message

## Data Preprocessing

In [5]:
# Build the complete dataset

gdp_raw = pd.read_excel('gdp.xlsx').set_axis(
    ['county', '2017', '2018', '2019', '2020', 'rank 2018', 'percent change 2018', 'percent change 2019', 'percent change 2020', 'rank 2020'], axis=1, inplace=False)
gdp_clean: pd.DataFrame = gdp_raw.drop(columns=['rank 2018', '2017', '2018', '2020',
                                                'rank 2020', 'percent change 2018', 'percent change 2019'], inplace=False).iloc[5:3222]

state_names = pd.read_csv('us-counties-2020.csv')['state'].unique()

counties_df = pd.read_csv('complete.csv')
counties_df['2019 raw GDP'] = np.nan  # iloc =5
counties_df['percent change 2020'] = np.nan  # iloc = 6

curr_state = None
for index, row in gdp_clean.iterrows():
    if row[0] in state_names:
        curr_state = row[0]
        continue
    else:
        row_index = counties_df.index[(counties_df['state'] == curr_state) & (
            counties_df['county'] == row[0])].tolist()
        counties_df.iloc[[row_index], [5]] = row[1]  # type: ignore
        counties_df.iloc[[row_index], [6]] = row[2]  # type: ignore


print(f"{len(counties_df['state'].unique())} states")
counties_df['2019 raw GDP'] = counties_df['2019 raw GDP'].astype('float64')
counties_df['percent change 2020'] = counties_df['percent change 2020'].astype('float64')
print(counties_df)


55 states
          county    state  cases  deaths  2020 population  2019 raw GDP  \
0        Autauga  Alabama   4190    48.0          58877.0     1540762.0   
1        Baldwin  Alabama  13601   161.0         233140.0     7134734.0   
2        Barbour  Alabama   1514    32.0          25180.0      729105.0   
3           Bibb  Alabama   1834    46.0          22223.0      380453.0   
4         Blount  Alabama   4641    63.0          59081.0      932215.0   
...          ...      ...    ...     ...              ...           ...   
3240  Sweetwater  Wyoming   2966    16.0          42158.0     3677972.0   
3241       Teton  Wyoming   2138     4.0          23347.0     2268742.0   
3242       Uinta  Wyoming   1558     7.0          20441.0      881052.0   
3243    Washakie  Wyoming    780    19.0           7658.0      349686.0   
3244      Weston  Wyoming    476     2.0           6809.0      322576.0   

      percent change 2020  
0                    -1.3  
1                    -2.1  
2    

In [6]:
encoded_state_names = pd.get_dummies(counties_df['state'])
counties_df = counties_df.drop(columns=['state'])\
                        .join(encoded_state_names)\
                        .dropna(axis='index', how='any')
print(counties_df)

          county  cases  deaths  2020 population  2019 raw GDP  \
0        Autauga   4190    48.0          58877.0     1540762.0   
1        Baldwin  13601   161.0         233140.0     7134734.0   
2        Barbour   1514    32.0          25180.0      729105.0   
3           Bibb   1834    46.0          22223.0      380453.0   
4         Blount   4641    63.0          59081.0      932215.0   
...          ...    ...     ...              ...           ...   
3240  Sweetwater   2966    16.0          42158.0     3677972.0   
3241       Teton   2138     4.0          23347.0     2268742.0   
3242       Uinta   1558     7.0          20441.0      881052.0   
3243    Washakie    780    19.0           7658.0      349686.0   
3244      Weston    476     2.0           6809.0      322576.0   

      percent change 2020  Alabama  Alaska  Arizona  Arkansas  ...  Tennessee  \
0                    -1.3        1       0        0         0  ...          0   
1                    -2.1        1       0   

## Building the model

In [7]:
# Setup some constants
DEFAULT_LEARNING_RATE: Final[float] = 0.1
DEFAULT_EPOCHS: Final[int] = 500

In [8]:
def BuildDefaultModel():
    SGD_optimizer: Final = tf.keras.optimizers.SGD(
        learning_rate=DEFAULT_LEARNING_RATE)
    lossFunction = tf.keras.losses.MeanSquaredError()

    model: keras.Sequential = keras.Sequential([
        keras.Input(shape=(59)),
        tf.keras.layers.Dense(
            units=30, activation=tf.nn.relu, name='hidden_layer_1'),
        tf.keras.layers.Dense(
            units=30, activation=tf.nn.relu, name='hidden_layer_2'),
        tf.keras.layers.Dense(
            units=15, activation=tf.nn.relu, name='hidden_layer_3'),
        tf.keras.layers.Dense(
            units=5, activation=tf.nn.relu, name='hidden_layer_4'),
        tf.keras.layers.Dense(units=1, activation=tf.nn.relu, name='output'),
    ], name='Default_COVID_Classifier')

    model.compile(loss=lossFunction, optimizer=SGD_optimizer,
                  metrics=['MeanSquaredError'])
    return model


defaultCovidClassifier = BuildDefaultModel()
print(defaultCovidClassifier.summary())


Model: "Default_COVID_Classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer_1 (Dense)      (None, 30)                1800      
                                                                 
 hidden_layer_2 (Dense)      (None, 30)                930       
                                                                 
 hidden_layer_3 (Dense)      (None, 15)                465       
                                                                 
 hidden_layer_4 (Dense)      (None, 5)                 80        
                                                                 
 output (Dense)              (None, 1)                 6         
                                                                 
Total params: 3,281
Trainable params: 3,281
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
input_attributes = counties_df.drop(
    columns=['percent change 2020', 'county'], inplace=False)

input_attributes['2019 raw GDP'] = input_attributes['2019 raw GDP'].astype(
    'float64')
input_attributes['cases'] = input_attributes['cases'].astype('int32')

input_attributes['Positivity Rate'] = input_attributes['cases'] / \
    input_attributes['2020 population']
input_attributes['Death Rate'] = input_attributes['deaths'] / \
    input_attributes['2020 population']

input_attributes.drop(columns=['cases', 'deaths'], inplace=True)

input_attributes['2019 raw GDP'] = MinMaxScaler().fit_transform(
    np.array(input_attributes['2019 raw GDP']).reshape(-1, 1))
input_attributes['2020 population'] = MinMaxScaler().fit_transform(
    np.array(input_attributes['2020 population']).reshape(-1, 1))
print(input_attributes)

      2020 population  2019 raw GDP  Alabama  Alaska  Arizona  Arkansas  \
0            0.005855      0.002164        1       0        0         0   
1            0.023301      0.010113        1       0        0         0   
2            0.002481      0.001011        1       0        0         0   
3            0.002185      0.000516        1       0        0         0   
4            0.005875      0.001300        1       0        0         0   
...               ...           ...      ...     ...      ...       ...   
3240         0.004181      0.005201        0       0        0         0   
3241         0.002298      0.003199        0       0        0         0   
3242         0.002007      0.001227        0       0        0         0   
3243         0.000727      0.000472        0       0        0         0   
3244         0.000642      0.000434        0       0        0         0   

      California  Colorado  Connecticut  Delaware  ...  Utah  Vermont  \
0              0         0

In [7]:
y_vals = MinMaxScaler().fit_transform(
    np.array(counties_df['percent change 2020']).reshape(-1, 1))
y_vals = y_vals.reshape(y_vals.shape[0])

In [8]:
X_Train, X_Test, y_Train, y_Test = train_test_split(
    input_attributes, y_vals, test_size=0.1, random_state=44)

print(X_Train.shape, y_Train.shape)

(2046, 59) (2046,)


In [9]:
# Compile and Train

startTime = time.time()

defaultCovidClassifier.fit(
    x=X_Train, y=y_Train,
    epochs=DEFAULT_EPOCHS,
    validation_data=(X_Test, y_Test))

endTime = time.time()
print(f'Training took {endTime-startTime} seconds')


Epoch 1/500
64/64 [==============================] - 1s 3ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0075 - val_mean_squared_error: 0.0075
Epoch 2/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.0073 - val_mean_squared_error: 0.0073
Epoch 3/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0072 - val_mean_squared_error: 0.0072
Epoch 4/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
Epoch 5/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0069 - val_mean_squared_error: 0.0069
Epoch 6/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
Epoch 7/500
64/64 [===============

In [10]:
predicted_y: np.ndarray = defaultCovidClassifier.predict(
    X_Test)  # raw continuous outputs

print(f'MSE of test set is: {mean_squared_error(predicted_y, y_Test)}')


MSE of test set is: 0.006762514170947118


In [11]:
def generateValidationData(inputDF: pd.DataFrame, expectedOutputs: np.ndarray,
                           trainIndexes: np.ndarray,
                           testIndexes: np.ndarray):
    X_Train = inputDF.iloc[trainIndexes]
    X_Test = inputDF.iloc[testIndexes]

    y_Train = expectedOutputs[trainIndexes]
    y_Test = expectedOutputs[testIndexes]

    return X_Train, X_Test, y_Train, y_Test


In [12]:
# Do 10-fold validation

KFolder = KFold(n_splits=10)
mseScores: list[float] = []
accuracyScores: list[float] = []

for trainIndexes, testIndexes in KFolder.split(input_attributes):
    X_Train, X_Test, y_Train, y_Test = generateValidationData(
        input_attributes, y_vals, trainIndexes, testIndexes)

    defaultCovidClassifier.fit(
        x=X_Train, y=y_Train,
        epochs=DEFAULT_EPOCHS,
        validation_data=(X_Test, y_Test))

    predicted_y: np.ndarray = defaultCovidClassifier.predict(X_Test)

    mseScores.append(mean_squared_error(y_Test, predicted_y))


Epoch 1/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 2/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 3/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 4/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 5/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0044 - val_mean_squared_error: 0.0044
Epoch 6/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0043 - mean_squared_error: 0.0043 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 7/500
64/64 [===============

In [13]:
print(f'MSE Loss: {mseScores}\nAverage MSE is: {np.average(mseScores)}')

MSE Loss: [0.004512871705139331, 0.0025231299277416494, 0.003177704128180342, 0.0036238820495055894, 0.002529873698385767, 0.007066750559344212, 0.0036164537178335634, 0.004594978051647303, 0.008314283767448312, 0.0030152532817427427]
Average MSE is: 0.004297518088696882


## Exercise 3

In [18]:
def BuildModel(numNodesLayer1=12, numNodesLayer2=3, learningRate=DEFAULT_LEARNING_RATE):
    SGD_optimizer: Final = tf.keras.optimizers.SGD(learning_rate=learningRate)
    lossFunction: Final = tf.keras.losses.MeanSquaredError()

    model: keras.Sequential = keras.Sequential([
        keras.Input(shape=(59)),
        tf.keras.layers.Dense(
            units=numNodesLayer1, activation=tf.nn.relu, name='hidden_layer_1'),
        tf.keras.layers.Dense(
            units=numNodesLayer2, activation=tf.nn.relu, name='hidden_layer_2'),
        tf.keras.layers.Dense(units=1, activation=tf.nn.relu, name='output'),
    ], name='COVID_Classifier')

    model.compile(loss=lossFunction, optimizer=SGD_optimizer,
                  metrics=['accuracy'])
    return model


wrappedCovidClassifier = KerasRegressor(build_fn=BuildModel)

# parameters passed to BuildModel(...)
param_grid = dict(
    nb_epoch=np.array(list(range(500, 600))),
    learningRate=np.array([0.1, 0.3]),
    numNodesLayer1=np.array([13, 23]),
    numNodesLayer2=np.array([7, 10]),
)

grid = GridSearchCV(estimator=wrappedCovidClassifier,
                    param_grid=param_grid, n_jobs=-1, cv=10)

# Extremely slow, do not run this repeatedly
# grid_result = grid.fit(X_Train, y_Train, epochs=2, validation_data=(X_Test, y_Test))


/var/folders/n9/fnc__vj96pbc8hshz5gbdl7r0000gn/T/ipykernel_55575/3527402640.py:20: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  wrappedCovidClassifier = KerasRegressor(build_fn=BuildModel)


building modelbuilding model
building model
building model
building model

building model
building model
building model
building model
building model
building model
building model
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
Epoch 1/2
58/58 [==============================] - 1s 11ms/step - loss: 0.0115 - accuracy: 5.4259e-04 - val_loss: 0.0114 - val_accuracy: 0.0044
Epoch 2/2
58/58 [==============================] - 1s 11ms/step - loss: 0.0091 - accuracy: 5.4289e-04 - val_loss: 0.0179 - val_accuracy: 0.0044
Epoch 2/2
58/58 [==============================] - 1s 11ms/step - loss: 0.0083 - accuracy: 5.4289e-04 - val_loss: 0.0129 - val_accuracy: 0.0044
Epoch 2/2
58/58 [==============================] - 1s 12ms/step - loss: 0.0089 - accuracy: 5.4289e-04 - val_loss: 0.0050 - val_accuracy: 0.0044
Epoch 2/2
58/58 [==============================] - 1s 12ms/step - loss: 0.0106 - accuracy: 0.0000e+00 - val_loss: 0.0038 - val_accurac

In [1]:
# Big reveal of best parameters
grid_result.best_params_

NameError: name 'grid_result' is not defined